# Model Training: Higgs Boson Classification

This notebook trains neural network models for Higgs boson classification.

In [ ]:
import sys
sys.path.append('../src')

import torch
import numpy as np
import matplotlib.pyplot as plt

from data_loader import load_higgs_data, get_data_loaders
from models import create_model
from trainer import Trainer
from visualization import plot_training_history, plot_roc_curve, plot_confusion_matrix, plot_prediction_distribution

%matplotlib inline

## Configuration

In [ ]:
# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Training configuration
config = {
    'batch_size': 128,
    'learning_rate': 0.001,
    'epochs': 50,
    'early_stopping_patience': 10,
    'model_type': 'simple'  # 'simple', 'standard', or 'deep'
}

## Load and Prepare Data

In [ ]:
# Load data
X_train, X_test, y_train, y_test = load_higgs_data(
    data_path='../data/HIGGS.csv',
    n_samples=50000,
    test_split=0.2,
    random_seed=42
)

# Create data loaders
train_loader, val_loader, test_loader = get_data_loaders(
    X_train, X_test, y_train, y_test,
    batch_size=config['batch_size'],
    val_split=0.1
)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")

## Create Model

In [ ]:
# Create model
input_dim = X_train.shape[1]
model = create_model(
    model_type=config['model_type'],
    input_dim=input_dim
)

print(f"Model: {config['model_type']}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"\nModel architecture:")
print(model)

## Train Model

In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    device=device,
    learning_rate=config['learning_rate']
)

# Train
history = trainer.train(
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=config['epochs'],
    early_stopping_patience=config['early_stopping_patience'],
    verbose=True
)

## Visualize Training History

In [ ]:
plot_training_history(
    history,
    save_path='../figures/training_history.png'
)

## Evaluate on Test Set

In [ ]:
# Evaluate
metrics = trainer.evaluate(test_loader)

print("Test Set Performance:")
print(f"  Accuracy: {metrics['accuracy']:.4f}")
print(f"  Precision: {metrics['precision']:.4f}")
print(f"  Recall: {metrics['recall']:.4f}")
print(f"  F1 Score: {metrics['f1_score']:.4f}")
print(f"\nConfusion Matrix:")
print(f"  True Positives: {metrics['tp']}")
print(f"  False Positives: {metrics['fp']}")
print(f"  True Negatives: {metrics['tn']}")
print(f"  False Negatives: {metrics['fn']}")

## Get Predictions

In [ ]:
# Get predictions on test set
model.eval()
y_pred_probs = []
y_pred_labels = []
y_true_all = []

with torch.no_grad():
    for data, target in test_loader:
        data = data.to(device)
        outputs = model(data)
        probs = outputs.cpu().numpy()
        y_pred_probs.extend(probs)
        y_pred_labels.extend((probs >= 0.5).astype(int))
        y_true_all.extend(target.numpy())

y_pred_probs = np.array(y_pred_probs).flatten()
y_pred_labels = np.array(y_pred_labels).flatten()
y_true_all = np.array(y_true_all).flatten()

## ROC Curve

In [ ]:
auc_score = plot_roc_curve(
    y_true_all,
    y_pred_probs,
    save_path='../figures/roc_curve.png'
)
print(f"AUC Score: {auc_score:.4f}")

## Confusion Matrix

In [ ]:
plot_confusion_matrix(
    y_true_all,
    y_pred_labels,
    save_path='../figures/confusion_matrix.png'
)

## Prediction Distribution

In [ ]:
plot_prediction_distribution(
    y_true_all,
    y_pred_probs,
    save_path='../figures/prediction_distribution.png'
)

## Save Model

In [ ]:
# Save model
model_path = f"../models/higgs_classifier_{config['model_type']}.pth"
trainer.save_model(model_path)